# Exploring Coincidences with a Cross-Verified database of Notable people

***Student:*** Lais Isabelle ALVES DOS SANTOS

## Libraries

In [41]:
# Libraries
import pandas as pd
from geopy.geocoders import Nominatim

In [45]:
# Instances
geolocator = Nominatim(user_agent="coincidences", timeout=10) 

## Data loading and pre-processing

In [83]:
df = pd.read_csv('../database/cross-verified-database.csv', 
                 encoding='latin-1',
                 delimiter=',',
                 usecols=['name', 'birth', 'death', 'level1_main_occ', 'gender', 'bplo1', 'dplo1', 'bpla1', 'dpla1', 'area1_of_rattachment']
                 )

In [85]:
# Replace some names to clarify the use of the dataset
df.rename(columns={"level1_main_occ": "domain", 
                   "bplo1": "birth_longitude", 
                   "dplo1": "death_longitude", 
                   "bpla1": "birth_latitude", 
                   "dpla1": "death_latitude",
                   "area1_of_rattachment": "country"}, inplace=True)

# Leave only notable people from 'Discovery/Science'
df = df[df.domain == 'Discovery/Science']

# Remove countries with names out of pattern
df = df[df.country != 'Old_(before_year_1990_AD)_Germany']

# Drop all the rows with nan values
df = df.dropna()

# Reset indices
df.reset_index(drop=True)

,birth,death,gender,domain,name,country,birth_longitude,death_longitude,birth_latitude,death_latitude
0,1907.0,1980.0,Female,Discovery/Science,Sylvia_Bayr-Klimpfinger,Austria,16.372499,16.372499,48.208332,48.208332
1,1935.0,2009.0,Male,Discovery/Science,Otto_Arndt_Liebisch,Germany,12.375000,9.866694,51.340000,52.499973
2,1929.0,2003.0,Male,Discovery/Science,Hans_ElsÃ¤sser,Germany,10.093611,8.710000,48.837223,49.412224
3,1912.0,1993.0,Male,Discovery/Science,Michael_de_Ferdinandy,Hungary,19.040833,-1.257778,47.498333,51.751945
4,1764.0,1847.0,Male,Discovery/Science,Imre_Festetics,Austria-Hungary,16.850000,16.551945,47.416668,47.381668
...,...,...,...,...,...,...,...,...,...,...
76565,1901.0,1937.0,Male,Discovery/Science,Lev_Shubnikov,Russia,30.316668,90.000000,59.950001,65.000000
76566,1720.0,1800.0,Male,Discovery/Science,Jean-Baptiste_Le_Roy,France,2.351389,2.351389,48.856945,48.856945
76567,1548.0,1606.0,Male,Discovery/Science,Johann_Jakob_RÃ¼eger,Switzerland,8.633860,8.633860,47.696529,47.696529
76568,1934.0,2004.0,Male,Discovery/Science,Erik_FlÃ¼gel,Austria,16.083334,11.004444,47.049999,49.596390


## Compute Unexpectedness

### Description complexity

In [ ]:
# Place of birth feature



### Causal complexity